<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/FileVital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [3]:
import pandas as pd
vital = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/vitalsign.csv')
vital.shape

(9213497, 7)

In [4]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].unique().tolist()

# Remove patient UIDs from visit['PatientUID'] that are not in the list
vital = vital[~vital['PatientUID'].isin(patient_uid_list)]
vital.shape

(8099274, 7)

In [5]:
# Specify the columns for unique value counting
columns_to_count = ['OrderItemName', 'ResultItemName']

# Count unique values in the specified columns
unique_counts = vital[columns_to_count].value_counts().reset_index()

In [6]:
unique_counts.columns = ['OrderItemName','ResultItemName', 'Count']

In [7]:
selected_items = ['Systolic BP', 'Diastolic BP', 'BMI', 'BSA']
filtered_items = unique_counts[unique_counts['ResultItemName'].isin(selected_items)]
filtered_items

,OrderItemName,ResultItemName,Count
0,Vital Signs Record,Systolic BP,1221173
1,Vital Signs Record,Diastolic BP,1220095
7,Vital Signs Record,BMI,558791
8,Vital Signs Record,BSA,558502


In [8]:
#filtered_items.to_excel('VitalFile.xlsx')

## **Slope**

> **ค่า x = ค่าจากวิสิท t ที่มาก่อน**




In [9]:
import pandas as pd
slope = pd.read_excel('/content/drive/MyDrive/Thesis Project Kidney/Assign/SummaryFile/LabSlope.xlsx')

In [10]:
slope

,PatientUID,PatientVisitUID,ResultValue,CWhen,datetime,TimeDiff,Slopes
0,2299,3640924,66.80,2559-11-11 09:40:06.053,2016-11-11 09:40:06.053,87,NaN
1,2299,4020970,67.20,2560-03-31 10:18:54.363,2017-03-31 10:18:54.363,140,0.002857
2,2299,4286612,69.20,2560-07-05 10:40:40.763,2017-07-05 10:40:40.763,96,0.020833
3,2299,4546471,63.60,2560-10-03 09:19:54.710,2017-10-03 09:19:54.710,89,-0.062921
4,2299,4801938,58.40,2561-01-04 09:03:28.847,2018-01-04 09:03:28.847,92,-0.056522
...,...,...,...,...,...,...,...
280078,3409733,10435209,49.13,2566-03-02 09:24:57.310,2023-03-02 09:24:57.310,83,-0.208313
280079,3432056,10235244,48.51,2566-01-09 12:31:53.220,2023-01-09 12:31:53.220,25,0.168400
280080,3432056,10306813,67.71,2566-01-27 12:22:43.000,2023-01-27 12:22:43.000,17,1.129412
280081,3432056,10408954,68.69,2566-02-24 21:05:12.747,2023-02-24 21:05:12.747,28,0.035000


In [11]:
data = pd.DataFrame(slope)
data = data[['PatientUID','PatientVisitUID','Slopes']]
data = data.dropna()
data

,PatientUID,PatientVisitUID,Slopes
1,2299,4020970,0.002857
2,2299,4286612,0.020833
3,2299,4546471,-0.062921
4,2299,4801938,-0.056522
5,2299,5051781,0.047619
...,...,...,...
280078,3409733,10435209,-0.208313
280079,3432056,10235244,0.168400
280080,3432056,10306813,1.129412
280081,3432056,10408954,0.035000


In [12]:
df = pd.DataFrame(data)

# Sort the DataFrame by PatientUID and PatientVisitUID
df = df.sort_values(by=['PatientUID', 'PatientVisitUID'])

# Create a new column to store the previous slope value
df['PreviousSlope'] = df['Slopes'].shift(1)

# Update the Slopes column with the PreviousSlope values
df['Slopes'] = df['PreviousSlope']

# Drop the PreviousSlope column
df = df.drop(columns=['PreviousSlope'])

In [13]:
data = df.dropna()

In [14]:
data

,PatientUID,PatientVisitUID,Slopes
2,2299,4286612,0.002857
3,2299,4546471,0.020833
4,2299,4801938,-0.062921
5,2299,5051781,-0.056522
6,2299,5296557,0.047619
...,...,...,...
280078,3409733,10435209,-0.655833
280079,3432056,10235244,-0.208313
280080,3432056,10306813,0.168400
280081,3432056,10408954,1.129412


In [15]:
vital['ResultItemName'].nunique()

18

# **TestWideTable**


In [16]:

# Initialize a dictionary to store the vital values
vital_dict = {}

# Iterate through unique ResultItemName values
for result_item in vital['ResultItemName'].unique():
    # Filter vital DataFrame for the current result_item
    subset_vital = vital[vital['ResultItemName'] == result_item]

    # Merge the subset_vital with data on PatientUID and PatientVisitUID
    merged_data = pd.merge(data, subset_vital[['PatientUID', 'PatientVisitUID', 'ResultValue', 'CWhen']], on=['PatientUID', 'PatientVisitUID'], how='left')

    # Rename the ResultValue column to match the result_item
    merged_data.rename(columns={'ResultValue': result_item}, inplace=True)

    # Update the vital_dict with the result_item as the key and merged_data as the value
    vital_dict[result_item] = merged_data

# Concatenate the dictionary values to get the final result
final_result = pd.concat(vital_dict.values(), axis=1)

# Drop duplicate columns (except for 'PatientUID', 'PatientVisitUID', and 'Slopes')
final_result = final_result.loc[:, ~final_result.columns.duplicated()]


# **solved the dup rows**


In [17]:

# Sort data by 'CWhen' in descending order within each patient's visit and keep the latest visit row
final_result.sort_values(by=['PatientUID', 'PatientVisitUID', 'CWhen'], ascending=[True, True, False], inplace=True)
final_result.drop_duplicates(subset=['PatientUID', 'PatientVisitUID'], keep='first', inplace=True)


In [18]:
vitalDF = final_result[['PatientUID','PatientVisitUID','Slopes','Systolic BP', 'Diastolic BP', 'BMI', 'BSA']]
# Drop rows with missing values in 'PatientUID', 'PatientVisitUID', and 'Slopes'
vitalDF = vitalDF.dropna(subset=['PatientUID', 'PatientVisitUID', 'Slopes'])

In [19]:
vitalDF.shape

(280081, 7)

In [20]:
data.shape

(280081, 3)

In [21]:
vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
1,2299.0,4286612.0,0.002857,129,82,21.94,1.41
2,2299.0,4546471.0,0.020833,146,81,20.27,1.38
3,2299.0,4801938.0,-0.062921,105,64,NaN,NaN
4,2299.0,5051781.0,-0.056522,111,58,20.49,1.38
5,2299.0,5296557.0,0.047619,125,65,20.89,1.4
...,...,...,...,...,...,...,...
289474,3409733.0,10435209.0,-0.655833,123,67,NaN,NaN
289475,3432056.0,10235244.0,-0.208313,134,78,NaN,NaN
289476,3432056.0,10306813.0,0.168400,140,66,NaN,NaN
289477,3432056.0,10408954.0,1.129412,146,NaN,NaN,NaN


## **Check Duplicate vitalDF**
    cuz there are the same visitID but in different Lab
    

In [22]:
# Check for duplicate rows in data
duplicates_data = data[data.duplicated(['PatientUID', 'PatientVisitUID'], keep=False)]

# Check for duplicate rows in vitalDF
duplicates_vitalDF = vitalDF[vitalDF.duplicated(['PatientUID', 'PatientVisitUID', 'Slopes'], keep=False)]


In [23]:
duplicates_vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA


    this is visitID that had dup value
    but we select only latest value

In [24]:
vital[vital['PatientVisitUID']==4286612]

,LabNo,PatientUID,PatientVisitUID,OrderItemName,ResultItemName,ResultValue,CWhen
338602,1,2299,4286612,Vital Signs Record,BSA,1.42,2560-07-05 09:00:18.093
338603,2,2299,4286612,Vital Signs Record,BSA,1.41,2560-07-05 14:29:33.277
338604,1,2299,4286612,Vital Signs Record,Systolic BP,138,2560-07-05 09:00:18.093
338605,2,2299,4286612,Vital Signs Record,Systolic BP,129,2560-07-05 14:29:33.277
338606,1,2299,4286612,Vital Signs Record,Waistline,72,2560-07-05 14:29:33.277
338607,1,2299,4286612,Vital Signs Record,Weight,48.6,2560-07-05 09:00:18.093
338608,2,2299,4286612,Vital Signs Record,Weight,48.7,2560-07-05 14:29:33.277
338753,1,2299,4286612,Vital Signs Record,BMI,21.6,2560-07-05 09:00:18.093
338754,2,2299,4286612,Vital Signs Record,BMI,21.94,2560-07-05 14:29:33.277
338755,1,2299,4286612,Vital Signs Record,Pulse Rate,76,2560-07-05 09:00:18.093


In [25]:
vitalDF[vitalDF['PatientVisitUID']==4286612.0]

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
1,2299.0,4286612.0,0.002857,129,82,21.94,1.41


In [26]:
vitalDF

,PatientUID,PatientVisitUID,Slopes,Systolic BP,Diastolic BP,BMI,BSA
1,2299.0,4286612.0,0.002857,129,82,21.94,1.41
2,2299.0,4546471.0,0.020833,146,81,20.27,1.38
3,2299.0,4801938.0,-0.062921,105,64,NaN,NaN
4,2299.0,5051781.0,-0.056522,111,58,20.49,1.38
5,2299.0,5296557.0,0.047619,125,65,20.89,1.4
...,...,...,...,...,...,...,...
289474,3409733.0,10435209.0,-0.655833,123,67,NaN,NaN
289475,3432056.0,10235244.0,-0.208313,134,78,NaN,NaN
289476,3432056.0,10306813.0,0.168400,140,66,NaN,NaN
289477,3432056.0,10408954.0,1.129412,146,NaN,NaN,NaN


In [29]:
vitalDF.to_excel('vitalDF.xlsx')